![pipeline](pictures/pictures.002.png)

# Text-Fabric from ETCBC

This notebook assembles the data from the ETCBC that is needed
to compile its datasets in text-fabric-format on GitHub.
Ultimately the data for the website [SHEBANQ](https://shebanq.ancient-data.org) will be
derived from these Text-Fabric-sources.

## Pipeline
This is **pipe 1** of the pipeline from ETCBC data to the website SHEBANQ.

A run of this pipe produces a data *version*.
It should be run whenever there are new or updated data sources present that affect the output data.
Since all input data is delivered in a GitHub repo, we have excellent machinery to 
work with versioning.

The pipe works by executing a series of programs, contained in GitHub repositories.
For each repository in the pipe, a series of notebooks will be executed.
See [script mode](https://github.com/ETCBC/pipeline/blob/master/README.md#operation) for 
details on how we call notebooks.

All this is specified in the configuration below.

### Core data

The core data is delivered by the ETCBC as `bhsa.mql.bz2` in 
the GitHub repo [bhsa](https://github.com/ETCBC/bhsa) in directory `source`.

This data will be converted by `tfFromMQL` in the `programs` directory.

The result of this action will be an updated Text-Fabric resource in its 
`tf/core` directory.

### Additional data

Researchers have contributed to the dataset, 
but not all that data is in the core.
They are typically in the repository where the research has been 
executed, and where the data is documented.

Before the pipe starts, these repos must be pulled.

## Continuous version
Version `c` acts as a *continuous* version. It will be overwritten
by new snapshots of the data on a regular basis.

We support the following workflow to carry out these updates:

1. make a new version called `_temp`. Note:
   * this choice of name prevents it to reach github, because `_temp` directories are in `.gitignore`;
   * after running this workflow, the version `_temp` already exists, this is not a problem;
2. put a new data snapshot in the `source/_temp` directory of the `bhsa` repo, add also data to the
   `source/_temp` directories of the other repos in the pipeline, as far as relevant;
3. run `good = runPipeline(pipeline, versions=['_temp'], force=True)`. Note:
   * we use `force=True` here, because then the old data in version `_temp` will be thoroughly overwritten;
4. if all went well run `copyVersion(pipeline, '_temp', 'c')`. This will overwrite all data directories
   in version `c` by the just created data directories in `temp`.

In [2]:
import os,sys,collections
from pipeline import runPipeline, copyVersion
from tf.fabric import Fabric

# Config

In [3]:
CORE_NAME = 'bhsa'

if 'SCRIPT' not in locals(): 
    SCRIPT = False
    DEFAULT_CORE_NAME = CORE_NAME
    DEFAULT_VERSION = 'c'

# Pipeline settings

Here all the nitty-gritty differences between versions are stated.

In [4]:
pipeline = dict(
    defaults = dict(
        CORE_NAME=CORE_NAME,
        VERSION=DEFAULT_VERSION,
        LANG_FEATURE='language',
        OCC_FEATURE='g_cons',
        LEX_FEATURE='lex',
        TEXT_FEATURE='g_word_utf8',
        TRAILER_FEATURE='trailer_utf8',
        DO_VOCALIZED_LEXEME=True,
        EXTRA_OVERLAP='',
        LEX_FORMATS='@fmt:lex-trans-plain={lex0} ',
        RENAME=(
            ('g_suffix', 'trailer'),
            ('g_suffix_utf8', 'trailer_utf8'),
        ),
    ),
    versions={
        '_temp': dict(),
        'c': dict(),
        '2017': dict(),
        '2016': dict(),
        '4b': dict(
                DO_VOCALIZED_LEXEME=False,
                EXTRA_OVERLAP='gloss nametype',
                LEX_FORMATS='@fmt:lex-trans-plain={lex} ',
            ),
        '4': dict(
                DO_VOCALIZED_LEXEME=False,
                EXTRA_OVERLAP='gloss nametype',
                LEX_FORMATS='@fmt:lex-trans-plain={lex} ',
        ),
        '3': dict(
                LANG_FEATURE='language',
                OCC_FEATURE='surface_consonants',
                LEX_FEATURE='lexeme',
                TEXT_FEATURE='text',
                TRAILER_FEATURE='suffix',
                LEX_FORMAT='@fmt:lex-trans-plain={lexeme} ',
            ),
    },
    repoOrder = '''
        bhsa
        phono
        valence
        parallels
    ''',
    repoConfig = dict(
        bhsa=(
            dict(
                task='coreData',
            ),
            dict(
                task='bookNames',
                omit={},
            ),
            dict(
                task='lexicon',
                omit={'3'},
            ),
            dict(
                task='paragraphs',
                 omit={'3', '4', '4b'},
            ),
            dict(
                task='ketivQere',
                omit={'3', '4', '4b'},
            ),
            dict(
                task='stats',
                omit={'4', '4b'},
            ),
        ),
        phono=(
            dict(
                task='phono',
                omit={'3', '4', '4b'},
            ),
        ),
        valence=(
            dict(
                task='enrich',
                omit={'3'},
            ),
            dict(
                task='flowchart',
                omit={'3'},
            ),
        ),
        parallels=(
            dict(
                task='parallels',
                omit={},
                params=dict(
                    FORCE_MATRIX=False,
                ),
            ),
        ),
    ),
    repoDataDirs = dict(
        bhsa      = 'source _temp tf shebanq',
        phono     = '_temp tf',
        valence   = 'source _temp tf shebanq',
        parallels = 'source _temp tf',
    ),
)

# Run the pipeline

To run everything from scratch:

```python
good = runPipeline(pipeline, versions=['3', '4', '4b', '2016', '2017', 'c'], force=True)
```

To make a new version called `temp`

```python
good = runPipeline(pipeline, versions=['_temp'], force=True) # a new candidate for 'c'
```

To copy the `_temp` version over to the continuous version `c`:

```python
if good:
    copyVersion(pipeline, '_temp', 'c')
```
This will copy all data, `source`, `_temp` and `tf` over from `_temp` to `c`.

If you want to create the node mappings between versions,
go to the versionMappings notebook in the BHSA repo and run it.

In [5]:
# good = runPipeline(pipeline, versions=['3', '4', '4b', '2016', '2017', 'c'], force=True)
good = runPipeline(pipeline, versions=['2017', '4', '4b', '2016'], force=True)


##############################################################################################
#                                                                                            #
#       0.00s Make version [2017]                                                            #
#                                                                                            #
##############################################################################################


**********************************************************************************************
*                                                                                            *
*       0.00s Make repo [bhsa]                                                               *
*                                                                                            *
**********************************************************************************************


---------------------------------------------

  0.21s 			feature txt (str) =def=  : node
  0.21s 			feature typ (str) =def= Unkn : node
  0.22s 			feature kind (str) =def= unknown : node
  0.22s 			feature rela (str) =def= NA : node
  0.22s 			feature mother_object_type (str) =def= clause : node
  0.22s 			feature dist_unit (str) =def= clause_atoms : node
  0.22s 		otype half_verse
  0.22s 			feature label (str) =def=  : node
  0.22s 		otype verse
  0.22s 			feature verse (int) =def= 0 : node
  0.22s 			feature chapter (int) =def= 0 : node
  0.23s 			feature label (str) =def=  : node
  0.23s 			feature book (str) =def= Genesis : node
  0.23s 		otype phrase_atom
  0.23s 			feature number (int) =def= 0 : node
  0.23s 			feature dist (int) =def= 0 : node
  0.24s 			feature distributional_parent (str) =def= 0 : edge
  0.24s 			feature mother (str) =def= 0 : edge
  0.24s 			feature functional_parent (str) =def= 0 : edge
  0.24s 			feature det (str) =def= NA : node
  0.24s 			feature typ (str) =def= VP : node
  0.24s 			feature rela (st

   |     0.40s T mother_object_type   to /Users/dirk/github/etcbc/bhsa/_temp/2017/tf
   |     0.76s T nme                  to /Users/dirk/github/etcbc/bhsa/_temp/2017/tf
   |     0.78s T nu                   to /Users/dirk/github/etcbc/bhsa/_temp/2017/tf
   |     2.31s T number               to /Users/dirk/github/etcbc/bhsa/_temp/2017/tf
   |     0.64s T otype                to /Users/dirk/github/etcbc/bhsa/_temp/2017/tf
   |     0.75s T pdp                  to /Users/dirk/github/etcbc/bhsa/_temp/2017/tf
   |     0.85s T pfm                  to /Users/dirk/github/etcbc/bhsa/_temp/2017/tf
   |     0.76s T prs                  to /Users/dirk/github/etcbc/bhsa/_temp/2017/tf
   |     0.77s T prs_gn               to /Users/dirk/github/etcbc/bhsa/_temp/2017/tf
   |     1.26s T prs_nu               to /Users/dirk/github/etcbc/bhsa/_temp/2017/tf
   |     1.04s T prs_ps               to /Users/dirk/github/etcbc/bhsa/_temp/2017/tf
   |     0.96s T ps                   to /Users/dirk/github/etcbc

|      5m 24s mother                    ... no changes
|      5m 24s mother_object_type        ... no changes
|      5m 24s nme                       ... no changes
|      5m 25s nu                        ... no changes
|      5m 25s number                    ... no changes
|      5m 27s oslots                    ... differences after the metadata
|      5m 28s 	line 1010820 OLD -->1,84,197,220,241,270,318,330,334,428,435 ...<--
|      5m 28s 	line 1010820 NEW --><empty><--
|      5m 28s 	line 1010821 OLD -->2,4662,27811,41331,48284,53077,66101,796 ...<--
|      5m 28s 	line 1010821 NEW --><empty><--
|      5m 28s 	line 1010822 OLD -->3,381,535,545,550,724,736,2126,2137,2148 ...<--
|      5m 28s 	line 1010822 NEW --><empty><--
|      5m 28s 	line 1010823 OLD -->4,26,34,42,50,60,81,97,127,142,162,176,1 ...<--
|      5m 28s 	line 1010823 NEW --><empty><--

|      5m 28s otext                     ... differences
|      5m 28s 	line      5 OLD -->@dateWritten=2017-10-10T10:55:37Z<--
|     

   |     1.07s T g_nme_utf8           from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.74s T g_pfm                from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.77s T g_pfm_utf8           from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.74s T g_prs                from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.88s T g_prs_utf8           from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.68s T g_uvf                from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.67s T g_uvf_utf8           from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.79s T g_vbe                from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.72s T g_vbe_utf8           from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.68s T g_vbs                from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     0.73s T g_vbs_utf8           from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     1.59s T g_voc_lex            from /Users/dirk/github/etcbc/bhsa/tf/2017
   |     2.52s T g_voc_lex_u

...

|     53m 51s 	Destination /Users/dirk/github/etcbc/valence/tf/2016/.tf/valence.tfx exists
|     53m 51s NOTE: repo seems up to date. Will be run because of "force=True"
True True
..............................................................................................
.     53m 51s Load the existing TF dataset                                                   .
..............................................................................................
This is Text-Fabric 3.1.1
Api reference : https://github.com/Dans-labs/text-fabric/wiki/Api
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

107 features found and 0 ignored
  0.00s loading features ...
   |     0.23s B lex_utf8             from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     0.18s B lex                  from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     0.01s B gloss                from /Users/dirk/github/etcbc

|     54m 43s 	    1 clauses with  3 infinitive objects
|     54m 43s 	   18 clauses with  2 infinitive objects
|     54m 43s 	 1192 clauses with  1 infinitive object
|     54m 43s 	68948 clauses with  0 infinitive objects
|     54m 43s 	 1211 clauses with  a infinitive object
..............................................................................................
.     54m 43s Determinig kind of complements                                                 .
..............................................................................................
|     54m 45s 	Done
|     54m 45s 	Phrases of kind C :  19300
|     54m 45s 	Phrases of kind L :  11681
|     54m 45s 	Phrases of kind I :   6016
|     54m 45s 	Total complements :  36997
|     54m 45s 	Total phrases     : 214665
..............................................................................................
.     54m 45s Checking enrichment logic                                                      .
.................

117 features found and 0 ignored
  0.00s loading features ...
   |     0.20s B lex_utf8             from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     0.14s B lex                  from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     0.00s B gloss                from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     0.13s B sp                   from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     0.14s B vs                   from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     0.25s B rela                 from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     0.24s B typ                  from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     0.07s B function             from /Users/dirk/github/etcbc/bhsa/tf/2016
   |     0.82s T valence              from /Users/dirk/github/etcbc/valence/tf/2016
   |     0.79s T predication          from /Users/dirk/github/etcbc/valence/tf/2016
   |     0.82s T grammatical          from /Users/dirk/github/etcbc/valence/tf/2016
   |     0.41s T original             

|     55m 38s 	10000 clauses
|     55m 41s 	20000 clauses
|     55m 45s 	30000 clauses
|     55m 48s 	40000 clauses
|     55m 51s 	47383 clauses
..............................................................................................
.     55m 51s Writing sense feature to TF                                                    .
..............................................................................................
   |     0.10s T sense                to /Users/dirk/github/etcbc/valence/_temp/2016/tf
..............................................................................................
.     55m 51s Check differences with previous version                                        .
..............................................................................................
|     55m 51s 	no features to add
|     55m 51s 	no features to delete
|     55m 51s 	1 features in common
|     55m 51s sense                     ... no changes
|     55m 51s Done
................

   |     0.05s T crossrefSET          to /Users/dirk/github/etcbc/parallels/_temp/2016/tf
|     56m 09s Compiled 31742 cross references into 6215 notes
|     56m 10s Generated 6215 notes
..............................................................................................
.     56m 10s Check differences with previous version                                        .
..............................................................................................
|     56m 10s 	no features to add
|     56m 10s 	no features to delete
|     56m 10s 	3 features in common
|     56m 10s crossref                  ... no changes
|     56m 10s crossrefLCS               ... no changes
|     56m 10s crossrefSET               ... no changes
|     56m 10s Done
..............................................................................................
.     56m 10s Deliver data set to /Users/dirk/github/etcbc/parallels/tf/2016                 .
...............................................

# Consolidate the continuous version
If you have run an update version called `_temp`, and all has went well
you can copy over the entire version (including its source and temp directories to `c`).
This will happen for all repos in the pipeline.

In [ ]:
#good = True
#good = False

if good:
    copyVersion(pipeline, '_temp', 'c')